In [1]:
#!pip install -qU "semantic-router[pinecone]==0.0.73"

# Syncing Routes with Pinecone Index

When using the `PineconeIndex`, our `RouteLayer` is stored in two places:

* We keep route layer metadata locally.
* Vectors alongside a backup of our metadata is stored remotely in Pinecone.

By storing some data locally and some remotely we achieve improved persistence and the ability to recover our local state if lost. However, it does come with challenges around keep our local and remote instances synchronized. Fortunately, we have [several synchronization options](https://docs.aurelio.ai/semantic-router/route_layer/sync.html). In this example, we'll see how to use these options to keep our local and remote Pinecone instances synchronized.

In [2]:
from semantic_router import Route

# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

In [3]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# get at platform.openai.com
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)

encoder = OpenAIEncoder(name="text-embedding-3-small")

For our `PineconeIndex` we do the exact same thing, ie we initialize as usual:

In [4]:
import os
from semantic_router.index.pinecone import PineconeIndex

# get at app.pinecone.io
os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)

pc_index = PineconeIndex(
    dimensions=1536,
    init_async_index=True,  # enables asynchronous methods, it's optional
    sync=None,  # defines whether we sync between local and remote route layers
    # when sync is None, no sync is performed
)

## RouteLayer

The `RouteLayer` class supports both sync and async operations by default, so we initialize as usual:

In [5]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=pc_index)

We can check our route layer and index information as usual:

In [6]:
rl.list_route_names()

['politics', 'chitchat']

In [7]:
len(rl.index)

0

Let's see if our local and remote instances are synchronized...

In [8]:
import time

# due to pinecone indexing latency we wait 3 seconds
time.sleep(3)
rl.index._read_hash()

hash_id: sr_hash#


ConfigParameter(field='sr_hash', value='f8f04794014c855bd68e283e64c57d8cc7a92f2ecd143386105de98f57c55e04', namespace='', created_at='2024-11-10T21:41:35.991948')

In [9]:
rl.is_synced()

hash_id: sr_hash#


True

It looks like everything is synced! Let's try deleting our local route layer, initializing it with just the politics route, and checking again.

In [10]:
del rl

rl = RouteLayer(encoder=encoder, routes=[politics], index=pc_index)
time.sleep(3)

hash_id: sr_hash#


2024-11-10 22:41:41 WARNING semantic_router.utils.logger Local and remote route layers were not aligned. Remote hash not updated. Use `RouteLayer.get_utterance_diff()` to see details.


Let's try `rl.is_synced()` again:

In [11]:
rl.is_synced()

hash_id: sr_hash#


False

We can use the `get_utterance_diff` method to see exactly _why_ our local and remote are not synced

In [16]:
rl.get_utterance_diff()

['chitchat: how are things going?', "chitchat: how's the weather today?", "chitchat: let's go to the chippy", 'chitchat: lovely weather today', 'chitchat: the weather is horrendous', "politics: don't you just hate the president", "politics: don't you just love the president", "politics: isn't politics the best thing ever", 'politics: they will save the country!', "politics: they're going to destroy this country!", "politics: why don't you tell me about your political opinions"]
["politics: don't you just hate the president", "politics: don't you just love the president", "politics: isn't politics the best thing ever", 'politics: they will save the country!', "politics: they're going to destroy this country!", "politics: why don't you tell me about your political opinions"]


['+ chitchat: how are things going?',
 "+ chitchat: how's the weather today?",
 "+ chitchat: let's go to the chippy",
 '+ chitchat: lovely weather today',
 '+ chitchat: the weather is horrendous',
 "  politics: don't you just hate the president",
 "  politics: don't you just love the president",
 "  politics: isn't politics the best thing ever",
 '  politics: they will save the country!',
 "  politics: they're going to destroy this country!",
 "  politics: why don't you tell me about your political opinions"]

---